In [83]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot as plt
from math import pi

import numpy as np
from numpy import random

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions


In [14]:
 y = model.predict (x_train)

In [88]:
def pm0(x, m , s ):
      return tf.math.exp(-0.5*((x-m)/s)**2);

m_data =  2
s_data =  1
cte_init = 1/(np.sqrt(2*pi)*s_data)

cte = cte_init

batch_size = 100
#def custom_loss(layer):

latent_dim =  5

x_input = randn(latent_dim * batch_size)
# reshape into a batch of inputs for the network
x_train = x_input.reshape(batch_size, latent_dim)
y_predit = model.predict (x_train)
    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
def nce_loss(y_true,y_pred):
        
    # first fit kde on y- to get pn 
    #y = model.predict (x_train)
    f = lambda y: tfd.Independent(tfd.Normal(loc=y, scale=np.std(y)))
    #n = y_pred.shape[0]
    n = batch_size
    pn = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
    probs=[1 / n] * n),
    components_distribution=f(y_predit))
    
    # 
    p_n = pn.prob( y_pred) 
    p_m = pm0(y_pred, m_data, s_data) # noise (generator) density 
    temp = tf.math.reduce_mean( (p_n/(cte*p_m+p_n)))
    print ("confusion " , temp)
    result = tf.math.reduce_mean( tf.math.log(p_n/(cte*p_m+p_n)))
   
    return result


    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
def nce_metric(y_predit):
        
    # first fit kde on y- to get pn 
    #y = model.predict (x_train)
    f = lambda y: tfd.Independent(tfd.Normal(loc=y, scale=np.std(y)))
    #n = y_pred.shape[0]
    n = batch_size
    pn = tfd.MixtureSameFamily(
    mixture_distribution=tfd.Categorical(
    probs=[1 / n] * n),
    components_distribution=f(y_predit))
    
    # 
    print(np.mean(y_predit))
    p_n = pn.prob( y_predit) 
    p_m = pm0(y_predit, m_data, s_data) # noise (generator) density 
    print("pn ", np.mean(p_n))
    print("p_m ", np.mean(p_m))
    temp = tf.math.reduce_mean( (p_n/(cte*p_m+p_n)))
    print ("confusion proba " , temp)
    return temp

In [11]:
print(model)

In [90]:
latent_dim = 5
n_outputs = 1
model = Sequential()
model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
model.add(Dense(n_outputs, activation='linear'))


model.compile(loss= nce_loss, optimizer='adam')


confusion  Tensor("loss_14/dense_35_loss/Mean:0", shape=(), dtype=float32)


In [89]:
n_epochs = 500
for i in range(n_epochs):

    x_input = randn(latent_dim * batch_size)
    # reshape into a batch of inputs for the network
    x_train = x_input.reshape(batch_size, latent_dim)
    y_train = np.ones(batch_size)
    model.train_on_batch(x_train, y_train)
    
    y_predit = model.predict (x_train)
    nce_metric(y_predit)

nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), d

nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), d

nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), d

nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), d

nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)
nan
pn  nan
p_m  nan
confusion proba  tf.Tensor(nan, shape=(), dtype=float32)


KeyboardInterrupt: 

In [65]:
x_train = x_input.reshape(batch_size, latent_dim)
y_predit = model.predict (x_train)
# Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
nce_metric(y_predit)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9999434>

In [80]:
pm0(0.016741264, 2, 1)

<tf.Tensor: shape=(), dtype=float32, numpy=0.13992375>

In [91]:
y_predit = model.predict (x_train)
print(np.mean(y_predit))

-1.8780168


In [92]:
pm0(-1.8780168, m_data,s_data)

<tf.Tensor: shape=(), dtype=float32, numpy=0.00054239994>